In [1]:
import pandas as pd
import re

WINDOW = 1
IS_RAN = False

overview_df = pd.read_excel("wi_overview_cleaned.xlsx")
overview_df['rapporteur'] = overview_df['rapporteur'].str.lower()
supporter_df = pd.read_excel("supporter_overview.xlsx")
supporter_df['rapporteur'] = supporter_df['rapporteur'].str.lower()

print(len(overview_df))
print(len(supporter_df))

8143
59087


In [2]:
overview_df['rapporteur'] = overview_df['rapporteur'].str.split(', ')
overview_df = overview_df.explode('rapporteur')
overview_df = overview_df.reset_index(drop=True)

supporter_df['rapporteur'] = supporter_df['rapporteur'].str.split(', ')
supporter_df = supporter_df.explode('rapporteur')
supporter_df = supporter_df.reset_index(drop=True)

overview_df.drop(overview_df[overview_df['rapporteur'].isna()].index, inplace=True)
supporter_df.drop(supporter_df[supporter_df['rapporteur'].isna()].index, inplace=True)
supporter_df.drop(supporter_df[supporter_df['supporting member'].isna()].index, inplace=True)
overview_df.drop(overview_df[overview_df['rapporteur'] == '-'].index, inplace=True)
supporter_df.drop(supporter_df[supporter_df['rapporteur'] == '-'].index, inplace=True)
supporter_df.drop(supporter_df[supporter_df['supporting member'] == '-'].index, inplace=True)

print(len(overview_df))
print(len(supporter_df))

7420
58832


In [7]:
display(supporter_df.head(1))
display(overview_df.head(1))

,unique_ID,release,ran,rapporteur,supporting member
3,460001,Rel-12,N,ericsson,china mobile


,Unique_ID,release,ran,sa,ct,rapporteur,WI rapporteur name,WI rapporteur e-mail,is_hyperlink,is_download,Impacted TSs and TRs,start,Created,last modif,finish,duration,is_complete
0,2,Rel-6,Y,N,N,alcatel-lucent,alcatel,Francois Courau,N,N,-,2003-12-31,2003-10-02,2004-07-08,2003-12-31,0,Y


## 1.1. Generate Dyad data (for Ucinet) - supporter to rapporteur

In [17]:
dyad_all = supporter_df[['release', 'ran', 'supporting member', 'rapporteur']]

if IS_RAN:
    dyad_all.drop(dyad_all[dyad_all['ran'] == 'N'].index, inplace=True)

for rel in range(12, (12 + 7 - WINDOW)):
    result_dyads = pd.DataFrame()
    
    for win in range(WINDOW):
        dyads = dyad_all[dyad_all['release'] == f'Rel-{rel + win}'][['supporting member', 'rapporteur']]
        result_dyads = pd.concat([result_dyads, dyads])
        
    if WINDOW > 1:
        result_dyads.to_excel(f'ucinet/dyad data/dyad_release_{rel}_to_{rel + WINDOW - 1}.xlsx', index=False)
#         print(f'release_{rel}_to_{rel + WINDOW - 1}: {len(result_dyads)}')
    else:
        result_dyads.to_excel(f'ucinet/dyad data/dyad_release_{rel}.xlsx', index=False)
#         print(f'release_{rel}: {len(result_dyads)}')